In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nbimporter
import utilities
from utilities import CBOW
from utilities import preprocess, prepareData, make_context_vector

Importing Jupyter notebook from utilities.ipynb


In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

#file = open('rural.txt')
#raw_text1 = file.read()
raw_text1 = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold."""

raw_text = preprocess(raw_text1)
data = prepareData(raw_text)

vocab = set(raw_text)
vocab_size = len(vocab)
print(vocab_size)

word_to_ix = {word: i for i, word in enumerate(vocab)}

print(len(data))

65
79


In [4]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, 2*CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [5]:
count = 0
for epoch in range(1000):
    total_loss = torch.Tensor([0])
    for context, target in data:

        context_var = make_context_vector(context, word_to_ix)
        
        model.zero_grad()

        log_probs = model(context_var)
        #print(log_probs)
        loss = loss_function(log_probs,  autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        loss.backward()
        optimizer.step()

        total_loss += loss.data
        count+=1
    if count%20==0:
        print("total loss: ",total_loss)
    losses.append(total_loss)
print(losses)

total loss:  
 289.6360
[torch.FloatTensor of size 1]

total loss:  
 249.2893
[torch.FloatTensor of size 1]

total loss:  
 209.2131
[torch.FloatTensor of size 1]

total loss:  
 168.4090
[torch.FloatTensor of size 1]

total loss:  
 128.8903
[torch.FloatTensor of size 1]

total loss:  
 94.5719
[torch.FloatTensor of size 1]

total loss:  
 67.8939
[torch.FloatTensor of size 1]

total loss:  
 48.7730
[torch.FloatTensor of size 1]

total loss:  
 35.6081
[torch.FloatTensor of size 1]

total loss:  
 26.5913
[torch.FloatTensor of size 1]

total loss:  
 20.3556
[torch.FloatTensor of size 1]

total loss:  
 15.9674
[torch.FloatTensor of size 1]

total loss:  
 12.8319
[torch.FloatTensor of size 1]

total loss:  
 10.5609
[torch.FloatTensor of size 1]

total loss:  
 8.8846
[torch.FloatTensor of size 1]

total loss:  
 7.6181
[torch.FloatTensor of size 1]

total loss:  
 6.6382
[torch.FloatTensor of size 1]

total loss:  
 5.8619
[torch.FloatTensor of size 1]

total loss:  
 5.2346
[torc

In [8]:
vectors = make_context_vector(vocab, word_to_ix)
#print(vectors.view(1,-1))
print(word_to_ix)
x = autograd.Variable(torch.LongTensor([word_to_ix["winter"]]))
print(x.size())
ten = autograd.Variable(torch.LongTensor([word_to_ix["shall"]]))
#print(ten.view(1,-1))
print(model.parameters)
vocab_var = make_context_vector(vocab,word_to_ix)
a = model.init_embedding(vocab_var)
print(a[0][:10])
print(a[0][10:20])
torch.save(model,'modelNGram.pth')

{'when': 0, 'beauti': 55, 'To': 1, 'make': 2, 'brow': 3, 'fair': 4, 'day': 5, 'worth': 6, 'cold': 7, 'couldst': 8, 'thriftless': 9, 'If': 10, 'ask': 12, 'trench': 13, 'thi': 15, 'warm': 19, 'count': 17, 'field': 18, 'sunken': 21, 'eye': 22, 'forti': 24, 'success': 16, 'proud': 27, 'small': 28, 'blood': 29, 'held': 30, 'art': 31, 'see': 34, 'made': 33, 'lusti': 56, 'where': 35, 'child': 26, 'thine': 32, 'answer': 37, 'sum': 44, 'use': 38, 'weed': 39, 'mine': 60, 'were': 41, 'dig': 42, 'shame': 43, 'gaze': 45, 'how': 48, 'lie': 46, 'within': 49, 'treasur': 20, 'thou': 50, 'winter': 51, 'will': 47, 'youth': 52, 'old': 53, 'totter': 14, 'new': 23, 'besieg': 54, 'shall': 36, 'excus': 57, 'say': 58, 'prais': 59, 'deep': 25, 'much': 40, 'and': 61, 'then': 64, 'liveri': 62, 'deserv': 63, 'prove': 11}
torch.Size([1])
<bound method Module.parameters of CBOW (
  (embeddings): Embedding(65, 10)
  (linear1): Linear (40 -> 128)
  (linear2): Linear (128 -> 65)
)>
Variable containing:
 0.6806
-1.0492


/usr/local/lib/python3.5/dist-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type CBOW. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
